In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/balbi/Downloads/pyMorfologik-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pymorfologik import Morfologik
from pymorfologik.parsing import ListParser
from pymorfologik.parsing import BaseParser
from pymorfologik.parsing import DictParser

import numpy as np
from __future__ import print_function
import string
from multiprocessing import cpu_count, Pool
import dask.dataframe as dd
from dask.multiprocessing import get
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getEmotionStatusOfText(result):
    if(result < 0):
        return 'negative'
    elif(result > 0):
        return 'positive'
    else:
        return 'neutral'

In [3]:
def lemmatisation_dictParser(text):
    parser = DictParser()
    stemmer = Morfologik()
    stemming = stemmer.stem([text], parser)
    print(stemming)
    words_list = list()
    for key, val in stemming.items():
        words_list.append(val[0].encode('utf8'))
    return words_list

In [4]:
emotions = pd.read_csv('emotionsTable-with-neutralEmotion.csv')
def detectingEmotionsInText(words_list):
    numberOfWords = len(words_list)
    numberOfWordsFound = 0
    result = 0
    emotions_list = list()
    if(numberOfWords > 0):
        emotionsCounter = 0
        for el in words_list: 
            if((emotions['word'] == el)).any():
                emotions_list.append(el)
                emotionsCounter += float(emotions.loc[emotions['word'] == el]['sentiment'].mean())
                numberOfWordsFound = numberOfWordsFound + 1
        result = emotionsCounter/numberOfWords
    return [result, numberOfWordsFound, emotions_list]

In [37]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = result_df.reset_index()

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    lemmatisation_text = lemmatisation_text.decode('utf8').lower()
    words_list = lemmatisation_text.encode('utf8').split() #lemmatisation_dictParser(lemmatisation_text.encode('utf8'))
    [result, number, emotions_list] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'emotionsList': emotions_list, 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('restaurant_reviews-without-lematisation.csv', encoding='utf-8', index=False)

In [ ]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('FilmWeb-movies_reviews.csv')
result = result.reset_index()

def calculateOneRow(index, emotion, rate, comment):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    lemmatisation_text = lemmatisation_text.decode('utf8').lower()
    words_list = lemmatisation_dictParser(lemmatisation_text.encode('utf8'))
    [result, number, emotions_list] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'emotionsList': emotions_list, 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('FilmWeb-movies_reviews-analysis-dictParser.csv', encoding='utf-8', index=False) 

In [5]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('opineo-productsReviews.csv')
result = result.reset_index()

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    words_list = lemmatisation_text.split()
    [result, number] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('opineo-productsReviews-without-lematisation.csv', encoding='utf-8', index=False) 

ValueError: Metadata inference failed in `lambda`.

Original error is below:
------------------------
ValueError('too many values to unpack', u'occurred at index 0')

Traceback:
---------
  File "/anaconda2/lib/python2.7/site-packages/dask/dataframe/utils.py", line 137, in raise_on_meta_error
    yield
  File "/anaconda2/lib/python2.7/site-packages/dask/dataframe/core.py", line 3264, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "<timed exec>", line 31, in <lambda>
  File "/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py", line 4877, in apply
    ignore_failures=ignore_failures)
  File "/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py", line 4973, in _apply_standard
    results[i] = func(v)
  File "<timed exec>", line 31, in <lambda>
  File "<timed exec>", line 12, in calculateOneRow


In [ ]:
result = pd.read_csv('mediaKrytyk-commentsRates.csv')

In [ ]:
com = pd.read_csv('CSV/FilmWeb-comments-analysis-neutralemotion-results-3.csv')

In [ ]:
len(com)

In [ ]:
com = com.drop('wordsFound', axis=1)

In [ ]:
result = pd.read_csv('restaurant_reviews.csv')

In [ ]:
test = lemmatisation_dictParser("Potężne rozczarowanie Jeśli czerpaliście przyjemność z lektury książki Clinea - w kinie będzie Was aż skręcać z zażenowania i niedowierzania że z porządnej fabularnie pozycji wyciśnięto coś na tyle marnego i wypłyconego Elementy które składają się na głębię każdej historii: psychologia postaci sensowne dialogi między nimi osadzenie owych bohaterów w szerszym społeczeństwie")

In [ ]:
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')
test 

In [ ]:
import csv

with open('tymczasowy.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(test)

In [ ]:
result = pd.read_csv('opineo-productsReviews.csv')

In [ ]:
index,comment, emotion, rate)

In [32]:
result = pd.read_csv('restaurant_reviews-analisis-all-2.csv')

In [33]:
len(result)

6919

In [34]:
result_df = pd.DataFrame({'comment': result['text'], 'emotion': result['emotion'], 'rate': result['rate']})

In [36]:
result_df

,comment,emotion,rate
0,"Wspaniala kamienica, wieczorem pieknie oswietl...",negative,3.0
1,"Przepiękny kadr, niesamowicie przemiła obsługa...",positive,5.0
2,"Piękne miejsce, urokliwy wystrój i rewelacyjna...",positive,4.0
3,W chwili w której byłem w tym Miejscu / zaparł...,positive,5.0
4,Obsługa restauracji od samego wejścia sprawia ...,positive,5.0
5,Nie pierwszy i nie ostatni raz byliśmy w tej r...,positive,5.0
6,Miejsce przepieknie przystrojone kwiatami! Atm...,positive,5.0
7,Dziękujemy za miła wizytę w restauracji i pole...,positive,4.0
8,"Piękny wystrój, wyjątkowa obsługa i przede wsz...",positive,5.0
9,Przebywaliśmy dość krótko na lunch biznesowym ...,negative,3.0
